In [ ]:
!pip install -q albumentations

In [ ]:
!pip install -q torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.6/962.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.5 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from google.colab import drive
from tqdm.notebook import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from collections import OrderedDict
import torch.optim as optim

In [ ]:

drive.mount('/content/drive')


IMAGE_DIR = '/content/drive/My Drive/Assignment3_Food_Dataset/images/'
MASK_DIR = '/content/drive/My Drive/Assignment3_Food_Dataset/masks/'
MODEL_SAVE_PATH = '/content/drive/My Drive/Assignment3_Food_Dataset/attention_unet_model.pth'

device = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4
LEARNING_RATE = 1e-4
NUM_EPOCHS = 40
IMG_HEIGHT = 256
IMG_WIDTH = 256


CLASS_RGB_VALUES = OrderedDict([
    ('Background', (0, 0, 0)), ('Rice(Plain)', (238, 65, 182)), ('Rice(mixed)', (153, 200, 92)),
    ('bread', (96, 128, 0)), ('Curry', (224, 127, 107)), ('Vegetable', (131, 13, 186)),
    ('Cutlet ', (87, 170, 44)), ('Mashed vegetable', (204, 57, 122)), ('egg', (96, 160, 128)),
    ('Sauce', (247, 206, 86)), ('Salad', (16, 0, 0)), ('Pickle', (156, 198, 121)),
    ('Lentil', (156, 168, 198)), ('Fried item', (124, 12, 84)),
    ('plate', (160, 160, 160))
])

CLASS_NAMES = list(CLASS_RGB_VALUES.keys())
NUM_CLASSES = len(CLASS_NAMES)

Mounted at /content/drive


# **Food Dataset Class**

In [ ]:
class FoodDataset(Dataset):
    def __init__(self, image_dir, mask_dir, filenames, class_rgb_values, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.filenames = filenames
        self.transform = transform
        self.class_rgb_values = class_rgb_values

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        filename = self.filenames[idx]
        image_path = os.path.join(self.image_dir, filename)
        mask_filename = os.path.splitext(filename)[0] + '.png'
        mask_path = os.path.join(self.mask_dir, mask_filename)

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(mask_path)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

        semantic_mask = np.zeros(mask.shape[:2], dtype=np.int64)
        for i, rgb in enumerate(self.class_rgb_values):
            equality = np.equal(mask, rgb)
            class_map = np.all(equality, axis=-1)
            semantic_mask[class_map] = i

        if self.transform:
            augmented = self.transform(image=image, mask=semantic_mask)
            image = augmented['image']
            semantic_mask = augmented['mask']


        return image, semantic_mask.long()

# **Attention U-Net Model**

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),

            nn.Dropout(0.3),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.conv(x)


class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttentionGate, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi


class AttentionUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=NUM_CLASSES):
        super(AttentionUNet, self).__init__()
        self.Conv1 = ConvBlock(in_channels, 64)
        self.Maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Conv2 = ConvBlock(64, 128)
        self.Maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Conv3 = ConvBlock(128, 256)
        self.Maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Conv4 = ConvBlock(256, 512)
        self.Maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Conv5 = ConvBlock(512, 1024)
        self.Up5 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.Att5 = AttentionGate(F_g=512, F_l=512, F_int=256)
        self.Up_conv5 = ConvBlock(1024, 512)
        self.Up4 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.Att4 = AttentionGate(F_g=256, F_l=256, F_int=128)
        self.Up_conv4 = ConvBlock(512, 256)
        self.Up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.Att3 = AttentionGate(F_g=128, F_l=128, F_int=64)
        self.Up_conv3 = ConvBlock(256, 128)
        self.Up2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.Att2 = AttentionGate(F_g=64, F_l=64, F_int=32)
        self.Up_conv2 = ConvBlock(128, 64)
        self.Conv_1x1 = nn.Conv2d(64, out_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x1 = self.Conv1(x); x2 = self.Maxpool1(x1); x2 = self.Conv2(x2)
        x3 = self.Maxpool2(x2); x3 = self.Conv3(x3); x4 = self.Maxpool3(x3)
        x4 = self.Conv4(x4); x5 = self.Maxpool4(x4); x5 = self.Conv5(x5)
        d5 = self.Up5(x5); a4 = self.Att5(g=d5, x=x4); d5 = torch.cat((a4, d5), dim=1); d5 = self.Up_conv5(d5)
        d4 = self.Up4(d5); a3 = self.Att4(g=d4, x=x3); d4 = torch.cat((a3, d4), dim=1); d4 = self.Up_conv4(d4)
        d3 = self.Up3(d4); a2 = self.Att3(g=d3, x=x2); d3 = torch.cat((a2, d3), dim=1); d3 = self.Up_conv3(d3)
        d2 = self.Up2(d3); a1 = self.Att2(g=d2, x=x1); d2 = torch.cat((a1, d2), dim=1); d2 = self.Up_conv2(d2)
        out = self.Conv_1x1(d2)
        return out


# **Data Augmentation and Dataloaders**

In [ ]:
train_transform = A.Compose([
    A.Resize(IMG_HEIGHT, IMG_WIDTH),
    A.Rotate(limit=40, p=0.7),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.4),
    A.Normalize(mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0)),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(IMG_HEIGHT, IMG_WIDTH),
    A.Normalize(mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0)),
    ToTensorV2(),
])

all_filenames = sorted(os.listdir(IMAGE_DIR))
train_val_files, test_files = train_test_split(all_filenames, test_size=0.15, random_state=42)
train_files, val_files = train_test_split(train_val_files, test_size=0.176, random_state=42) # 0.176 of 85% is ~15% of total

train_dataset = FoodDataset(IMAGE_DIR, MASK_DIR, train_files, list(CLASS_RGB_VALUES.values()), transform=train_transform)
val_dataset = FoodDataset(IMAGE_DIR, MASK_DIR, val_files, list(CLASS_RGB_VALUES.values()), transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)


**Instantiate Model, Loss, Optimizer, and Scheduler**

In [ ]:
model = AttentionUNet(in_channels=3, out_channels=NUM_CLASSES).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scaler = torch.cuda.amp.GradScaler(enabled=(device=="cuda"))

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 'min', patience=5, factor=0.1, verbose=True
)

/tmp/ipython-input-8-228013908.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=="cuda"))
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


# **Training Loop and Metrics Evaluation**

In [ ]:
def check_metrics(loader, model, loss_fn, device="cuda"):
    model.eval()
    num_correct, num_pixels, total_iou = 0, 0, 0
    total_loss = 0.0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            preds = model(x)
            total_loss += loss_fn(preds, y).item()
            preds_labels = torch.argmax(preds, dim=1)
            num_correct += (preds_labels == y).sum()
            num_pixels += torch.numel(preds_labels)

            # Calculate IoU per batch
            for cls in range(NUM_CLASSES):
                pred_inds = (preds_labels == cls)
                target_inds = (y == cls)
                intersection = (pred_inds & target_inds).sum()
                union = (pred_inds | target_inds).sum()
                # Add a small epsilon to avoid division by zero
                total_iou += (intersection.float() / (union.float() + 1e-6))

    pixel_accuracy = (num_correct/num_pixels)*100

    mean_iou = total_iou / (len(loader.dataset) * NUM_CLASSES)
    avg_loss = total_loss / len(loader)

    model.train()
    return avg_loss, pixel_accuracy, mean_iou


best_val_iou = -1.0

for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0.0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")

    for batch_idx, (data, targets) in enumerate(loop):
        data, targets = data.to(device), targets.to(device)

        with torch.cuda.amp.autocast(enabled=(device=="cuda")):
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # Backward pass
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_train_loss = train_loss / len(train_loader)


    val_loss, val_pixel_acc, val_mean_iou = check_metrics(val_loader, model, loss_fn, device=device)


    scheduler.step(val_loss)

    print(
        f"Epoch {epoch+1}: \n"
        f"  Train Loss = {avg_train_loss:.4f} \n"
        f"  Val Loss   = {val_loss:.4f} \n"
        f"  Val Acc    = {val_pixel_acc:.2f}% \n"
        f"  Val mIoU   = {val_mean_iou:.4f}"
    )

    if val_mean_iou > best_val_iou:
        best_val_iou = val_mean_iou
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"==> New best model saved with mIoU: {best_val_iou:.4f}")

print("\n--- Training Finished ---")
print(f"Best validation mIoU achieved: {best_val_iou:.4f}")
print(f"Model saved to: {MODEL_SAVE_PATH}")


Epoch 1/40:   0%|          | 0/35 [00:00<?, ?it/s]

/tmp/ipython-input-9-1309963110.py:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


Epoch 1: 
  Train Loss = 2.3637 
  Val Loss   = 2.2632 
  Val Acc    = 42.94% 
  Val mIoU   = 0.0173
==> New best model saved with mIoU: 0.0173


Epoch 2/40:   0%|          | 0/35 [00:00<?, ?it/s]

Epoch 2: 
  Train Loss = 1.9305 
  Val Loss   = 1.8940 
  Val Acc    = 51.51% 
  Val mIoU   = 0.0284
==> New best model saved with mIoU: 0.0284


...

Epoch 38: 
  Train Loss = 0.9437 
  Val Loss   = 1.0842 
  Val Acc    = 65.71% 
  Val mIoU   = 0.0480


Epoch 39/40:   0%|          | 0/35 [00:00<?, ?it/s]

Epoch 39: 
  Train Loss = 0.9502 
  Val Loss   = 1.0330 
  Val Acc    = 67.97% 
  Val mIoU   = 0.0510


Epoch 40/40:   0%|          | 0/35 [00:00<?, ?it/s]

Epoch 40: 
  Train Loss = 0.9032 
  Val Loss   = 0.9850 
  Val Acc    = 69.34% 
  Val mIoU   = 0.0549
==> New best model saved with mIoU: 0.0549

--- Training Finished ---
Best validation mIoU achieved: 0.0549
Model saved to: /content/drive/My Drive/Assignment3_Food_Dataset/attention_unet_model.pth
